# Proximity data to Area of Prediction

This notebook contains the processes that interpolate proximity to certain amenities to the are of prediction.

## Import libraries

In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import aup

## Data download/read and preprocessing

### Area of analysis

In [2]:
aoa_gdf = gpd.read_file("../../data/processed/prediccion_uso_suelo/Distrito_Zona5.geojson")
print(aoa_gdf.shape)
aoa_gdf.head(2)

(1, 4)


,id,area,perimeter,geometry
0,0,3.456908e+07,22034.335455,"POLYGON ((662437.93 2284457.02, 662421.516 228..."


### Area of prediction with variables

In [3]:
gdf = gpd.read_file('../../data/processed/prediccion_uso_suelo/test/area_of_prediction_roads.gpkg')
print(gdf.shape)
gdf.head(2)

(88061, 46)


,uso_suelo,area_m2,full_plus_code,CVEGEO,bld_area_m2,block_area_m2,pred_area_m2,pred_area_pct,bld_pred_area_pct,pobtot,...,service_distance,living_street_distance,primary_distance,trunk_distance,pedestrian_distance,cycleway_distance,unclassified_distance,motorway_distance,steps_distance,geometry
0,Sin dato,67.279056,75GRMGQW+89PG,1412000011206018,12.438638,278416.454431,67.279056,0.000242,0.184881,0,...,11.962829,51.20766,3329.005414,87.628973,2935.411316,1968.027137,673.709507,374.060135,1584.676445,"MULTIPOLYGON (((-103.45405 20.68833, -103.4541..."
1,Sin dato,45.355545,75GRMGQW+8CFP,1412000011206018,4.482118,278416.454431,45.355545,0.000163,0.098822,0,...,11.962829,51.20766,3329.005414,87.628973,2935.411316,1968.027137,673.709507,374.060135,1584.676445,"MULTIPOLYGON (((-103.4539 20.68833, -103.454 2..."


### Proximity data

In [4]:
buffer = aoa_gdf.to_crs("EPSG:6372").buffer(100)
buffer = gpd.GeoDataFrame(geometry = buffer)
buffer = buffer.to_crs("EPSG:4326")
buffer

,geometry
0,"POLYGON ((-103.4407 20.65148, -103.44079 20.65..."


In [5]:
table = 'proximity_v2_23_point'
schema = 'prox_analysis'

prox_nodes = aup.gdf_from_polygon(buffer, schema, table)

print(prox_nodes.shape)
prox_nodes.head(2)

(9575, 69)


,osmid,denue_preescolar,denue_preescolar_15min,denue_primaria,denue_primaria_15min,denue_secundaria,denue_secundaria_15min,clues_primer_nivel,clues_primer_nivel_15min,denue_guarderias,...,denue_museos,denue_museos_15min,denue_bibliotecas,denue_bibliotecas_15min,denue_centrocultural,denue_centrocultural_15min,x,y,geometry,city
0,28791518,7.341088,6,7.777907,4,26.270394,0,8.583527,9,10.816726,...,29.825454,0,24.933597,0,88.667714,0,-103.402947,20.674933,POINT (-103.40295 20.67493),Guadalajara
1,273146454,2.454897,4,4.260768,1,15.860206,0,3.041695,11,17.525910,...,39.703464,0,17.089605,0,81.541184,0,-103.406217,20.671926,POINT (-103.40622 20.67193),Guadalajara


Filter proximity elements

In [6]:
cols = ['osmid','denue_primaria','denue_primaria_15min',
       'denue_abarrotes','denue_abarrotes_15min','denue_peluqueria',
       'denue_peluqueria_15min','denue_lavanderia','denue_lavanderia_15min',
       'clues_primer_nivel','clues_primer_nivel_15min','geometry']
prox_nodes = prox_nodes[cols].copy()

print(prox_nodes.shape)
prox_nodes.head(2)

(9575, 12)


,osmid,denue_primaria,denue_primaria_15min,denue_abarrotes,denue_abarrotes_15min,denue_peluqueria,denue_peluqueria_15min,denue_lavanderia,denue_lavanderia_15min,clues_primer_nivel,clues_primer_nivel_15min,geometry
0,28791518,7.777907,4,4.427983,9,5.178137,46,6.701719,7,8.583527,9,POINT (-103.40295 20.67493)
1,273146454,4.260768,1,3.563373,6,2.454897,36,9.127368,8,3.041695,11,POINT (-103.40622 20.67193)


## Proximity to area of analysis

Set CRS

In [7]:
prox_nodes = prox_nodes.to_crs("EPSG:6372")
gdf = gdf.to_crs("EPSG:6372")

Create centroids with unique id

In [8]:
if 'fid' not in gdf.columns:
    gdf = gdf.reset_index().rename(columns={'index':'fid'})

In [9]:
gdf_cnt = gdf[['fid','geometry']].copy()
gdf_cnt['geometry'] = gdf_cnt.centroid

Interpolate proximity data

In [10]:
for col in cols:
    if (col != 'osmid') and (col != 'geometry'):
        int_vals = aup.interpolate_at_points(prox_nodes.centroid.x,
                                         prox_nodes.centroid.y, 
                                         prox_nodes[col],
                                         gdf_cnt.geometry.x, 
                                         gdf_cnt.geometry.y,
                                        power=2,
                                        search_radius=300)
        
        gdf_cnt[col] = int_vals
        print('Finished processing',col)
        

Finished processing denue_primaria
Finished processing denue_primaria_15min
Finished processing denue_abarrotes
Finished processing denue_abarrotes_15min
Finished processing denue_peluqueria
Finished processing denue_peluqueria_15min
Finished processing denue_lavanderia
Finished processing denue_lavanderia_15min
Finished processing clues_primer_nivel
Finished processing clues_primer_nivel_15min


Merge data

In [11]:
gdf = gdf.merge(gdf_cnt.drop(columns=['geometry']),
               on='fid')
print(gdf.shape)
gdf.head(2)

(88061, 57)


,fid,uso_suelo,area_m2,full_plus_code,CVEGEO,bld_area_m2,block_area_m2,pred_area_m2,pred_area_pct,bld_pred_area_pct,...,denue_primaria,denue_primaria_15min,denue_abarrotes,denue_abarrotes_15min,denue_peluqueria,denue_peluqueria_15min,denue_lavanderia,denue_lavanderia_15min,clues_primer_nivel,clues_primer_nivel_15min
0,0,Sin dato,67.279056,75GRMGQW+89PG,1412000011206018,12.438638,278416.454431,67.279056,0.000242,0.184881,...,21.292222,0.686290,11.815648,3.201121,12.873068,3.446521,13.698224,1.478983,18.924816,0.357128
1,1,Sin dato,45.355545,75GRMGQW+8CFP,1412000011206018,4.482118,278416.454431,45.355545,0.000163,0.098822,...,19.025558,0.888095,11.066837,3.917225,12.427027,4.012734,13.493960,1.680444,16.927238,0.467758


## Save

In [12]:
gdf.to_file('../../data/processed/prediccion_uso_suelo/test/area_of_prediction_prox.gpkg')